In [1]:
import pandas as pd
import numpy as np
import json

In [233]:
#считываем данные 
data=pd.read_csv('data.csv')
day=pd.read_csv('context_day.csv')
place=pd.read_csv('context_place.csv')

In [234]:
#выбираем пользователя для расчета
variant = 38
userID= variant - 1
data.iloc[userID]

Unnamed: 0    User 38
 Movie 1            3
 Movie 2           -1
 Movie 3            1
 Movie 4            4
 Movie 5            2
 Movie 6            3
 Movie 7            4
 Movie 8           -1
 Movie 9            5
 Movie 10           3
 Movie 11           3
 Movie 12          -1
 Movie 13           3
 Movie 14           2
 Movie 15           4
 Movie 16           3
 Movie 17           1
 Movie 18           4
 Movie 19           2
 Movie 20           1
 Movie 21           5
 Movie 22           4
 Movie 23           3
 Movie 24          -1
 Movie 25           1
 Movie 26           4
 Movie 27           3
 Movie 28          -1
 Movie 29          -1
 Movie 30           4
Name: 37, dtype: object

In [235]:
#удаляем стоблбец с именами пользователей
data = data.drop(data.columns[0], axis = 1)

In [236]:
#находим метрику сходства с каждым пользователем
def sim(v):
    resultList = {}
    for u in range(data.shape[0]):
        num = 0
        vdenum = 0
        udenum = 0
        if (u != v):        
            uv = data.iloc[[u, v]]
            uv = uv.transpose()
            for i,movie in uv.iterrows():
                if ((movie[u] != -1) & (movie[v] != -1 )):
                    num += movie[u] * movie[v]
                    udenum += pow(movie[u], 2)
                    vdenum += pow(movie[v], 2)
                    res = round(num / ((math.sqrt(udenum) * math.sqrt(vdenum))), 3)
        resultList[u] = res
        #сортируем результат и выбираем четырех ближайших соседей
        sim = list(resultList.items())
        sim.sort(key = lambda i: i[1], reverse = True)
        sim = sim[:4]
    return sim

In [237]:
simk = sim(userID)
simk

[(25, 0.932), (21, 0.924), (5, 0.915), (11, 0.91)]

In [238]:
#функция для поиска средней оценки пользователя
def avg(user):
    count = 0
    mark = 0
    for i in range(1, data.shape[1]):
        if data.iloc[user][i] != -1:
            count += 1
            mark += data.iloc[user][i]
    res = round(mark/count,3)
    return res

In [239]:
avg(userID)

3.0

In [240]:
#с помощью метрики косинуса находим предполагаемую оценку пользователя для фильма
def getMark(u, movie):
    rates = {}
    num = 0
    denum = 0
    if data.iloc[u, movie] == -1:
        for v in simk:
                if data.iloc[v[0], movie] != -1:
                    num += v[1]*(data.iloc[v[0], movie]-avg(v[0]))
                    denum += v[1] 
    res = round(avg(u)+num/denum, 3)
    return res        

In [241]:
#получаем результат для всех неоцененных фильмов
def firstRes():
    result = []
    for i in range(1, data.shape[1]):
        if data.iloc[userID, i] == -1:
            result.append("Movie "+str(i)+": "+ str(getMark(userID, i)))
    return result

In [242]:
firstRes()

['Movie 1: 3.983',
 'Movie 7: 3.576',
 'Movie 11: 3.169',
 'Movie 23: 4.003',
 'Movie 27: 2.675',
 'Movie 28: 3.435']

In [243]:
#записываем результат в файл
def writeJson(uderID, firstTask, fileName='result.json'):
    result = {"user": variant,
              "1": firstTask,
              }
    with open(fileName, 'w') as f:
        json.dump(result, f, indent=10)

In [244]:
writeJson(userID, firstRes())